In [175]:
from autograd import make_jvp, jacobian
from autograd.differential_operators import make_jvp_reversemode
import autograd.numpy as np

**Objective**: Test jacobian calculation in python

First define the function

Autograd does not support assignment to index, so construct $\frac{dy}{dt}$ using stack instead of indexing

In [176]:
def f(x1,x2,x3,x4,x5,x6):
    # State is { x,y,theta, vx,vy,omega }
    dydt = []
    dydt.append( x4**2 + 2*x6 )
    dydt.append( 5*x5 )
    dydt.append( x4 + 2*x5 + 3*x6 )
    dydt.append( x1**2 + 2*x2 )
    dydt.append( x2**3 + 4*x3 )
    dydt.append( 0 )
    
    return np.stack(dydt)

Test that the calculation is what we expect

In [177]:
x = np.ones(6)
res1 = []
res2 = []
for i in range(6):
    res1.append( jacobian(f, argnum=i)(*x) )
    res2.append( make_jvp(f, argnum=i)(*x)(1)[1] )
res1 = np.stack(res1, axis=1)
res2 = np.stack(res2, axis=1)
print('jacobian:\n', res1)
print('make_jvp:\n',res2)

jacobian:
 [[0. 0. 0. 2. 0. 2.]
 [0. 0. 0. 0. 5. 0.]
 [0. 0. 0. 1. 2. 3.]
 [2. 2. 0. 0. 0. 0.]
 [0. 3. 4. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
make_jvp:
 [[0. 0. 0. 2. 0. 2.]
 [0. 0. 0. 0. 5. 0.]
 [0. 0. 0. 1. 2. 3.]
 [2. 2. 0. 0. 0. 0.]
 [0. 3. 4. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [197]:
make_jvp_reversemode(f, argnum=0)(1.0, 1.0, 1.0, 1.0, 1.0, 1.0)([1,0,0,0,0,0])

IndexError: tuple index out of range

## Compare speed of `Jacobian` vs `make_jvp`

make_jvp is expected to be much faster because it is a fast forward-mode Jacobian

### `Jacobian`

In [179]:
%%timeit
x = np.zeros(6)
J = [ jacobian(f, argnum=i) for i in range(6) ]
for t in range(1000):
    res = []
    for i in range(6):
        res.append( J[i](*x) )
    res = np.stack(res).T

2.98 s ± 31.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### `make_jvp`

In [180]:
%%timeit
x = np.zeros(6)
J = [ make_jvp_reversemode(f, argnum=i) for i in range(6) ]
for t in range(1000):
    res = []
    for i in range(6):
        res.append( J[i](*x)(1)[1] )
    res = np.stack(res).T

TypeError: 'int' object is not subscriptable

### Conclusion
`make_ivp` is around 70x faster than `jacobian`